In [74]:
import pandas as pd
import numpy as np
from collections import defaultdict
from surprise import Reader
from surprise import KNNBasic
from surprise import Dataset
from surprise.model_selection import cross_validate

In [211]:
#Reading the anime file which would be used for testing the trained model
anime = pd.read_csv('anime.csv')

In [212]:
check = pd.read_csv('anime.csv')
check.set_index('anime_id',inplace = True) #This is for displaying the title based on the anime-id predicted by out model

In [215]:
#dropping off some columns
file = anime.drop(['episodes','members','name','genre','type'], axis = 1)

In [216]:
#Reading the training data
rating = pd.read_csv('rating.csv')

In [217]:
df = rating.loc[0:50000] #Selecting a portion of data_set to avoid memory error

In [218]:
dz = df.drop(['anime_id','rating'], axis = 1) # selecting user_id from the rating data and merging it with the anime_data to form a new test data
dz = dz
.loc[0:12293]

In [219]:
file['user_id'] = dz

In [220]:
file #file = test_data

,anime_id,rating,user_id
0,32281,9.37,1
1,5114,9.26,1
2,28977,9.25,1
3,9253,9.17,1
4,9969,9.16,1
...,...,...,...
12289,9316,4.15,159
12290,5543,4.28,159
12291,5621,4.88,159
12292,6133,4.98,159


In [221]:
# Construct reader
reader = Reader(rating_scale=(1, 5))

# Generate surprise Dataset
data = Dataset.load_from_df(df[['user_id', 'anime_id', 'rating']], reader)

test_data = Dataset.load_from_df(file[['user_id', 'anime_id', 'rating']], reader)

In [222]:
# Set all data as training set
trainset = data.build_full_trainset()
testset = test_data.build_full_trainset()

# Build and train an algorithm.

sim_options = {
               'user_based': True  # compute  similarities between items
}

algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [223]:
user_id = 13;
anime_id = 177;

In [224]:
pred = algo.predict(user_id, anime_id, verbose=True)

user: 13         item: 177        r_ui = None   est = 1.00   {'actual_k': 21, 'was_impossible': False}


In [225]:
testset = testset.build_anti_testset()
predictions = algo.test(testset)


In [237]:
print(predictions[0:10])

[Prediction(uid=1, iid=1842, r_ui=nan, est=3.217630432765621, details={'actual_k': 7, 'was_impossible': False}), Prediction(uid=1, iid=6336, r_ui=nan, est=5, details={'actual_k': 7, 'was_impossible': False}), Prediction(uid=1, iid=20651, r_ui=nan, est=2.2033199309788727, details={'actual_k': 10, 'was_impossible': False}), Prediction(uid=1, iid=1210, r_ui=nan, est=2.0738017232008636, details={'actual_k': 40, 'was_impossible': False}), Prediction(uid=1, iid=338, r_ui=nan, est=5, details={'actual_k': 3, 'was_impossible': False}), Prediction(uid=1, iid=13759, r_ui=nan, est=1, details={'actual_k': 40, 'was_impossible': False}), Prediction(uid=1, iid=6547, r_ui=nan, est=1, details={'actual_k': 40, 'was_impossible': False}), Prediction(uid=1, iid=5081, r_ui=nan, est=1.0972699602287912, details={'actual_k': 40, 'was_impossible': False}), Prediction(uid=1, iid=32867, r_ui=nan, est=5, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}), Prediction(uid=1, iid=264, r_ui=nan,

In [236]:
# This block copied from Surprise documentation at
# http://surprise.readthedocs.io/en/stable/FAQ.html#how-to-get-the-top-n-recommendations-for-each-user

def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    if uid == 2:
        print(uid, [check.loc[iid] for (iid, _) in user_ratings]) #Here the model is referring to the anime_id on the check data-set for displaying the title

2 [name                              Kimi no Na wa.
genre       Drama, Romance, School, Supernatural
type                                       Movie
episodes                                       1
rating                                      9.37
members                                   200630
Name: 32281, dtype: object, name                         Fullmetal Alchemist: Brotherhood
genre       Action, Adventure, Drama, Fantasy, Magic, Mili...
type                                                       TV
episodes                                                   64
rating                                                   9.26
members                                                793665
Name: 5114, dtype: object, name                                                 Gintama°
genre       Action, Comedy, Historical, Parody, Samurai, S...
type                                                       TV
episodes                                                   51
rating                      

The code above shows the top ten movie predictions for user_2 with all the details displayed

In [231]:
#Evaluate accuracy
# MAE = mean absolute error, RMSE = root mean squared error
cross_validate(algo, data, measures=['MAE', 'RMSE'], cv=3, verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE, RMSE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
MAE (testset)     3.2548  3.2568  3.2254  3.2457  0.0143  
RMSE (testset)    3.6114  3.6127  3.5858  3.6033  0.0124  
Fit time          0.06    0.07    0.08    0.07    0.01    
Test time         0.86    0.91    0.89    0.89    0.02    


{'test_mae': array([3.25482967, 3.25678985, 3.22543174]),
 'test_rmse': array([3.61135514, 3.61274823, 3.58576223]),
 'fit_time': (0.06279563903808594, 0.06881570816040039, 0.08283615112304688),
 'test_time': (0.8597018718719482, 0.9115936756134033, 0.8896505832672119)}

 2 -The Algorithm displayed a MemoryError: Unable to allocate 40.3 GiB for an array with shape (73515, 73515) and data type float64. This is because windows hardware does not allow computation for the magnitude of the data being processed at a time.
-This was Solved by selecting only part of the data for training the model
 3 - The files can be merged by selecting the anime_id as index_col of a similar data_set and referring to that based on the predictions from the model.